In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_epochs = 3
batch_size = 64
learning_rate = 0.001
num_classes = 10

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=10):  # Update to the number of classes in MNIST
        super(ModifiedAlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),  # Update input channels to 1
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
model = ModifiedAlexNet(num_classes).to(device)
model.apply(weights_init)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_model(model, criterion, optimizer, train_loader, num_epochs):
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = correct / total * 100
        return accuracy

In [ ]:
train_model(model, criterion, optimizer, train_loader, num_epochs)

In [ ]:
baseline_accuracy = test_model(model, test_loader)
print(f"Baseline Accuracy: {baseline_accuracy:.2f}%")

In [ ]:
# Pruning rates for evaluation
pruning_rates = [0.2, 0.5, 0.8]

for prune_rate in pruning_rates:
        # Prune nodes using LDA
    def prune_nodes_LDA(model, prune_ratio):
        # Extracting weights
        weights = torch.cat([param.view(-1) for param in model.parameters()]).cpu().detach().numpy()

        # Generating pseudo-labels
        num_samples = len(weights)
        pseudo_labels = np.arange(num_samples) % 2

        # Print weights and pseudo-labels for debugging
        print("Weights before LDA pruning:", weights)
        print("Pseudo-labels for LDA:", pseudo_labels)

        # Applying LDA
        lda = LDA(n_components=1)
        lda.fit(weights.reshape(-1, 1), pseudo_labels)
        explained_variance_ratio = np.sum(lda.explained_variance_ratio_)

        # Print explained variance for debugging
        print("Explained Variance Ratio (LDA):", explained_variance_ratio)

        node_threshold = np.percentile(np.abs(weights), prune_ratio * 100)

        # Print node threshold for debugging
        print("Node Threshold (LDA):", node_threshold)

        # Prune nodes
        for param in model.parameters():
            param.data[torch.abs(param.data) < node_threshold] = 0

        return explained_variance_ratio

    explained_variance_ratio_nodes_lda = prune_nodes_LDA(model, prune_rate)
    accuracy_nodes_pruned_lda = test_model(model, test_loader)
    print(f"LDA Pruning Rate: {prune_rate}, Accuracy after LDA pruning: {accuracy_nodes_pruned_lda:.2f}%")
    print(f"Explained Variance Ratio (LDA): {explained_variance_ratio_nodes_lda:.2f}")

    # Prune connections using LDA
    def prune_connections_LDA(model, prune_ratio):
        # Extracting weights
        weights = torch.cat([param.view(-1) for param in model.parameters()]).cpu().detach().numpy()

        # Generating pseudo-labels
        num_samples = len(weights)
        pseudo_labels = np.arange(num_samples) % 2

        # Print weights and pseudo-labels for debugging
        print("Weights before LDA pruning:", weights)
        print("Pseudo-labels for LDA:", pseudo_labels)

        # Applying LDA
        lda = LDA(n_components=1)
        lda.fit(weights.reshape(-1, 1), pseudo_labels)
        explained_variance_ratio = np.sum(lda.explained_variance_ratio_)

        # Print explained variance for debugging
        print("Explained Variance Ratio (LDA):", explained_variance_ratio)

        weight_threshold = np.percentile(np.abs(weights), prune_ratio * 100)

        # Print node threshold for debugging
        print("weight Threshold (LDA):", weight_threshold)

        # Prune connections
        for param in model.parameters():
            param.data[torch.abs(param.data) < weight_threshold] = 0

        return explained_variance_ratio

    #explained_variance_ratio_connections_lda = prune_connections_LDA(model, prune_rate)
    #accuracy_connections_pruned_lda = test_model(model, test_loader)
    #print(f"LDA Pruning Rate: {prune_rate}, Accuracy after pruning connections: {accuracy_connections_pruned_lda:.2f}%")
    #print(f"Explained Variance Ratio (Connections - LDA): {explained_variance_ratio_connections_lda:.2f}")

    # Fine-tuning
    def fine_tune_model(model, criterion, optimizer, train_loader, num_epochs):
        model.train()
        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(train_loader):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i+1) % 100 == 0:
                    print ('Fine-tune Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                           .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

    fine_tune_epochs = 1
    fine_tune_model(model, criterion, optimizer, train_loader, num_epochs=fine_tune_epochs)

    accuracy_after_fine_tune_lda = test_model(model, test_loader)
    print(f"Accuracy after fine-tuning (LDA): {accuracy_after_fine_tune_lda:.2f}%")